In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Optional, Any

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
@dataclass
class Points:
    """
    2D points with x and y coordinates.
    """
    array: np.ndarray
        
    @property
    def x(self) -> np.ndarray:
        return self.array[:, 0]
        
    @property
    def y(self) -> np.ndarray:
        return self.array[:, 1]
    
    def __len__(self):
        return self.array.shape[0]
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, **kwargs) -> Any:
        if ax is None:
            ax = plt.gca()
        return ax.scatter(self.x, self.y, *args, **kwargs)

class Cell(ABC):
    """
    Abstract class for a cell.
    """
    @abstractmethod
    def sample(self, n: int) -> Points:
        """
        Sample uniformly n points within the current cell.
        """
        pass
    
    @abstractmethod
    def draw(self, ax: Optional[plt.Axes] = None, *args, **kwargs):
        """
        Draw the cell structure on the given axes.
        """
        pass
    
@dataclass
class Circle(Cell):
    radius: float = 1.0
    center: np.ndarray = np.array([0., 0.])
        
    def __post_init__(self):
        self.center = np.asarray(self.center).reshape(1, 2)
    
    def sample(self, n: int) -> Points:
        def sample_in_circle(n):
            xy = np.random.rand(n, 2)
            xy = (xy - .5) * (2 * self.radius)
            r = np.linalg.norm(xy, axis=1)
            
            return xy[r <= self.radius, :]
        
        n2 = n << 1
        xy = sample_in_circle(n2)
        
        while xy.shape[0] < n:
            xy = sample_in_circle(n2)
            
        return Points(array=xy[:n, :] + self.center)
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, fill=False, **kwargs):
        if ax is None:
            ax = plt.gca()
        circle = plt.Circle(self.center.flatten(), self.radius, *args, fill=fill, **kwargs)
        return ax.add_patch(circle)
    
@dataclass
class Hexagon(Cell):
    radius: float = 1.0
    center: np.ndarray = np.array([0., 0.])
    rotation: float = 0.0
        
    def __post_init__(self):
        self.center = np.asarray(self.center).reshape(1, 2)
      
    @property
    def corners_centered_array(self) -> np.ndarray:
        zero = 0.
        radi = self.radius
        s3_2 = np.sqrt(3) * .5 * self.radius
        ra_2 = self.radius * .5
        array = np.array([
            [+s3_2, +ra_2],
            [+zero, +radi],
            [-s3_2, +ra_2],
            [-s3_2, -ra_2],
            [+zero, -radi],
            [+s3_2, -ra_2],
        ])
        return array
    
    @property
    def rotation_array(self) -> np.ndarray:
        cos = np.cos(self.rotation)
        sin = np.sin(self.rotation)
        
        return np.array([
            [+cos, -sin],
            [+sin, +cos],
        ])
    
    @property
    def corners_centered(self) -> Points:
        array = self.corners_centered_array
        return Points(array=array)
    
    @property
    def corners_array(self) -> np.ndarray:
        array = self.corners_centered_array
        array = (self.rotation_array @ array.T).T
        return array + self.center
        
    @property
    def corners(self) -> Points:
        array = self.corners_array
        return Points(array=array)
    
    def sample(self, n: int) -> Points:
        def sample_in_hexagon(n):
            """
            We check that points are in hexagon using
            the 'cross product trick'.
            """
            xy = np.random.rand(n, 2)
            xy = (xy - .5) * (2 * self.radius)
            corners = self.corners_centered_array
            
            # Vectors from points (samples) to each corner
            v1 = - xy[:,:,None] + corners.T[None,:,:]
            v2 = - xy[:,:,None] + np.roll(corners, 1, axis=0).T[None,:,:]
            
            cross = np.cross(v1, v2, axis=1)
            
            # Sum is 6 (or -6) if all the signs are the same
            s = np.sum(np.sign(cross), axis=1)
            index = np.abs(s) == 6
            
            return xy[index, :]
        
        n2 = n << 1
        xy = sample_in_hexagon(n2)
        
        while xy.shape[0] < n:
            xy = sample_in_hexagon(n2)
            
        xy = (self.rotation_array @ xy.T).T
            
        return Points(array=xy[:n, :] + self.center)
    
    def draw(self, ax: Optional[plt.Axes] = None, *args, fill=False, **kwargs):
        if ax is None:
            ax = plt.gca()
        hexagon = plt.Polygon(self.corners_array, *args, fill=fill, **kwargs)
        return ax.add_patch(hexagon)
    
def received_power(points: Points, alpha: float = 2.0) -> np.ndarray:
    r = np.linalg.norm(points.array, axis=0)
    return 1 / r**alpha

In [ ]:
cell = Circle(radius=3)

points = cell.sample(1000)

cell.draw()
points.draw()

In [ ]:
radii = np.linspace(1, 10)
n = 10000

cell_type = Circle

for alpha in [2., 2.5, 3.]:
    powers = []
    stds = []
    for radius in radii:
        cell = cell_type(radius=radius)
        power = received_power(points=cell.sample(n), alpha=alpha)
        powers.append(np.mean(power))
        stds.append(np.std(power))
        
    plt.errorbar(radii, powers, yerr=stds, label=f"alpha = {alpha}")
    
plt.yscale("log")
plt.legend()